In [1]:
import pandas

In [2]:
a = pandas.read_csv("6M Partition/1820.csv")
a.head()

,textID,#words,genre,year,title,author,Publication information,Library of Congress classification (NF),FIXED
0,7562,"45,611",FIC,1820,The poetical works,"Trumbull, John, 1750-1831",NaN,NaN,n
1,8735,"12,558",FIC,1820,The Mountain Torrent,"Judah, Samuel B. H. (Samuel Benjamin Helbert),...",NaN,NaN,n
2,8900,"3,363",FIC,1820,The Irish Exiles in America,"Selden, Almira",NaN,NaN,n
3,8901,"1,303",FIC,1820,Lady Jane Gray and Lord Guilford Dudley,"Selden, Almira",NaN,NaN,n
4,8902,"3,040",FIC,1820,Naomi,"Selden, Almira",NaN,NaN,n


In [3]:
genre = ["FIC", "NF", "MAG"]
years = [1820, 1830, 1840, 1850, 1860, 1870, 1880, 1890, 1900, 1910, 1920, 1930, 1940, 1950, 1960, 1970, 1980, 1990, 2000]
verb = ['make', 'take', 'give']

In [4]:
for i in genre:
    print(i)

FIC
NF
MAG


In [10]:
f=open("fullSelect.sql", "a+")
for v in verb:
    for i in years:
        a = pandas.read_csv("6M Partition/" + str(i) + ".csv")
        for j in genre:
            sqlQuery = ""
            sqlSource = ""
            sqlQuery = "\n" + "--" + str(i) + j + v + "\n"
            b = a.loc[a['genre'] == j]
            for row in b.iterrows():
                sqlSource = sqlSource + "s.textid = " + str(row[1][0]) + " or "
            sqlSource = sqlSource[:-3]
            sqlQuery = """
COPY (
SELECT count(*), lemma from (
SELECT c.w2 FROM wordlines as c, lexicon as l , sources as s
where l.lemma like \'""" + v + "\'" + """
and c.w1 = l.wid 
and c.source = s.textid
and s.year BETWEEN """ + str(i) + " and " + str(i + 9) + """
and (
""" + sqlSource + ")" + """
UNION ALL """ + """
SELECT c.w3 FROM wordlines as c, lexicon as l , sources as s
where l.lemma like \'""" + v + "\'" + """
and c.w1 = l.wid 
and c.source = s.textid
and s.year BETWEEN """ + str(i) + " and " + str(i + 9) + """
and ( """ + sqlSource + ")" + """
UNION ALL """ + """
SELECT c.w4 FROM wordlines as c, lexicon as l , sources as s
where l.lemma like \'""" + v + "\'" + """
and c.w1 = l.wid 
and c.source = s.textid
and s.year BETWEEN """ + str(i) + " and " + str(i + 9) + """
and ( """ + sqlSource + ")" + """
UNION ALL """ + """
SELECT c.w5 FROM wordlines as c, lexicon as l , sources as s
where l.lemma like \'""" + v + "\'" + """
and c.w1 = l.wid 
and c.source = s.textid
and s.year BETWEEN """ + str(i) + " and " + str(i + 9) + """
and ( """ + sqlSource + ")" + """
UNION ALL """ + """
SELECT c.w6 FROM wordlines as c, lexicon as l , sources as s
where l.lemma like \'""" + v + "\'" + """
and c.w1 = l.wid 
and c.source = s.textid
and s.year BETWEEN """ + str(i) + " and " + str(i + 9) + """
and ( """ + sqlSource + ")" + """ a, lexicon b where 
(
b.pos like '%nn@%' or b.pos like '%nn\%%' or b.pos like 'nn'
or b.pos like '%nn1%' or b.pos like '%nn2%'
) 
and a.w2 = b.wid group by b.lemma order by count(*) desc 
) TO """ + "\'/var/lib/postgresql/" + v + str(i) + j + ".csv\' CSV HEADER;"
            
            f.write(sqlQuery)
f.close()